In [77]:
#!-*- coding: utf-8 -*-

In [74]:
import pandas as pd
from sklearn.model_selection import cross_val_score
import numpy as np
from collections import Counter

In [9]:
classificacoes = pd.read_csv('dados/emails.csv')
classificacoes.head()

,email,classificacao
0,"Se eu comprar cinco anos antecipados, eu ganho...",1
1,O exercício 15 do curso de Java 1 está com a r...,2
2,Existe algum curso para cuidar do marketing da...,3
3,"Gostaria de renovar antecipadamente meu plano,...",1
4,O vídeo não está travando as vezes no meu nave...,2


In [29]:
textosPuros = classificacoes['email']
textosQuebrados = textosPuros.str.lower().str.split(' ')

In [76]:
dicionario = set()

for lista in textosQuebrados:
    dicionario.update(lista)

totaldepalavras = len(dicionario)
# print(dicionario)

tuplas = list(zip(dicionario, range(totaldepalavras)))

tradutor = {palavra:indice for palavra,indice in tuplas}  

def vetorizar_texto(texto, tradutor):
    vetor = [0] * len(tradutor)

    for palavra in texto:
        if palavra in tradutor:
            posicao = tradutor[palavra]
            vetor[posicao] += 1
            
    return vetor


texto = textosQuebrados[0]


vetoresdetexto = [vetorizar_texto(texto, tradutor) for texto in textosQuebrados]
marcas = classificacoes['classificacao'] 

X = np.array(vetoresdetexto)
Y = np.array(marcas.tolist())

porcentagem_de_treino = 0.8

tamanho_de_treino = int(porcentagem_de_treino * len(Y))
tamanho_de_validacao = len(Y) - tamanho_de_treino

treino_dados = X[0:tamanho_de_treino]
treino_marcacoes = Y[0:tamanho_de_treino]

validacao_dados = X[tamanho_de_treino:]
validacao_marcacoes = Y[tamanho_de_treino:]

def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k)
    taxa_de_acerto = np.mean(scores)
    msg = "Taxa de acerto do {0}: {1}".format(nome, taxa_de_acerto)
    print(msg)
    return taxa_de_acerto

def teste_real(modelo, validacao_dados, validacao_marcacoes):
    resultado = modelo.predict(validacao_dados)

    acertos = resultado == validacao_marcacoes

    total_de_acertos = sum(acertos)
    total_de_elementos = len(validacao_marcacoes)

    taxa_de_acerto = 100.0 * total_de_acertos / total_de_elementos

    msg = "Taxa de acerto do vencedor entre os dois algoritmos no mundo real: {0}".format(taxa_de_acerto)
    print(msg)

resultados = {}

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state = 0, max_iter=10000))
resultadoOneVsRest = fit_and_predict("OneVsRest", modeloOneVsRest, treino_dados, treino_marcacoes)
resultados[resultadoOneVsRest] = modeloOneVsRest

from sklearn.multiclass import OneVsOneClassifier
modeloOneVsOne = OneVsOneClassifier(LinearSVC(random_state = 0, max_iter=10000))
resultadoOneVsOne = fit_and_predict("OneVsOne", modeloOneVsOne, treino_dados, treino_marcacoes)
resultados[resultadoOneVsOne] = modeloOneVsOne

from sklearn.naive_bayes import MultinomialNB
modeloMultinomial = MultinomialNB()
resultadoMultinomial = fit_and_predict("MultinomialNB", modeloMultinomial, treino_dados, treino_marcacoes)
resultados[resultadoMultinomial] = modeloMultinomial

from sklearn.ensemble import AdaBoostClassifier
modeloAdaBoost = AdaBoostClassifier()
resultadoAdaBoost = fit_and_predict("AdaBoostClassifier", modeloAdaBoost, treino_dados, treino_marcacoes)
resultados[resultadoAdaBoost] = modeloAdaBoost

print(resultados)

maximo = max(resultados)
vencedor = resultados[maximo]

print("Vencerdor: ")
print(vencedor)

vencedor.fit(treino_dados, treino_marcacoes)

teste_real(vencedor, validacao_dados, validacao_marcacoes)

acerto_base = max(Counter(validacao_marcacoes).values())
taxa_de_acerto_base = 100.0 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base: %f" % taxa_de_acerto_base)

total_de_elementos = len(validacao_dados)
print("Total de teste: %d" % total_de_elementos)

Taxa de acerto do OneVsRest: 0.7166666666666667
Taxa de acerto do OneVsOne: 0.625
Taxa de acerto do MultinomialNB: 0.7333333333333334
Taxa de acerto do AdaBoostClassifier: 0.4666666666666666
{0.7166666666666667: OneVsRestClassifier(estimator=LinearSVC(max_iter=10000, random_state=0)), 0.625: OneVsOneClassifier(estimator=LinearSVC(max_iter=10000, random_state=0)), 0.7333333333333334: MultinomialNB(), 0.4666666666666666: AdaBoostClassifier()}
Vencerdor: 
MultinomialNB()
Taxa de acerto do vencedor entre os dois algoritmos no mundo real: 77.77777777777777
Taxa de acerto base: 44.444444
Total de teste: 9
